# Sequence Labeling: Shallow Parsing

## Objectives

- Understanding: 
    - relation between Sequence Labeling and Shallow Parsing
    - IOB Notation
    - Joint Segmentation and Classification
    - Feature Engineering
- Learning how to:
    - use Named Entity Recognition in 
        - spacy
        - NLTK
    - train, test, and evaluate Conditional Random Fields models
    - perform feature engineering with CRF

### Recommended Reading
- Dan Jurafsky and James H. Martin. [__Speech and Language Processing__ (SLP)](https://web.stanford.edu/~jurafsky/slp3/) (3rd ed. draft)
- Steven Bird, Ewan Klein, and Edward Loper. [__Natural Language Processing with Python__ (NLTK)](https://www.nltk.org/book/)
- Conditional Random Fields
    - Lafferty et al. (2001) [Conditional Random Fields: Probabilistic Models for Segmenting and Labeling Sequence Data](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.26.803&rep=rep1&type=pdf) (__original paper__)
    - Sutton & McCallum's [An Introduction to Conditional Random Fields](https://homepages.inf.ed.ac.uk/csutton/publications/crftutv2.pdf)
    - Edwin Chen's [Introduction to Conditional Random Fields](http://blog.echen.me/2012/01/03/introduction-to-conditional-random-fields/)
    - Michael Collin's [Log-Linear Models, MEMMs, and CRFs](http://www.cs.columbia.edu/~mcollins/crf.pdf)

### Covered Material
- SLP
    - [Chapter 8: Sequence Labeling for Parts of Speech and Named Entities](https://web.stanford.edu/~jurafsky/slp3/8.pdf) 
- NLTK 
    - [Chapter 7: Extracting Information from Text](https://www.nltk.org/book/ch07.html) 

### Requirements

- [spaCy](https://spacy.io/)
- [NLTK](https://www.nltk.org/)
- [`conll.py`](https://github.com/esrel/LUS/) (in `src` folder)
- [CRFsuite](http://www.chokkan.org/software/crfsuite/)
    - [python-crfsuite](https://python-crfsuite.readthedocs.io) python binding to `CRFsuite`.
    - [sklearn-crfsuite](https://sklearn-crfsuite.readthedocs.io) `python-crfsuite` wrapper providing API similar to `scikit-learn`
    - you need to install both `python_crfsuite` and `sklearn_crfsuite` to use `sklearn_crfsuite`

## 1. Sequence Labeling and Shallow Parsing

Below are some examples of NLP tasks that Sequence Labeling is applied to as one of the methods.

The scenario when members of a sequence are mapped to higher order units (i.e. grouped together `[['a'],['b','c']]`) and assigned a category) is known as __shallow parsing__.

- [Part-of-Speech Tagging](https://en.wikipedia.org/wiki/Part-of-speech_tagging)
- [Shallow Parsing](https://en.wikipedia.org/wiki/Shallow_parsing) (Chunking)
    - [Phrase Chunking](https://en.wikipedia.org/wiki/Phrase_chunking)
    - [Named-Entity Recognition](https://en.wikipedia.org/wiki/Named-entity_recognition) 
    - [Semantic Role Labeling](https://en.wikipedia.org/wiki/Semantic_role_labeling)
    - Dependency [Parsing](https://en.wikipedia.org/wiki/Parsing) 
    - Discourse Parsing
    - (Natural/Spoken) __Language Understanding__: Concept Tagging/Entity Extraction

### 1.1. The General Setting for Sequence Labeling

- Create __training__ and __testing__ sets by tagging a certain amount of text by hand
    - i.e. map each word in corpus to a tag
- Train tagging model to extract generalizations from the annotated __training__ set
- Evaluate the trained tagging model on the annotated __testing__ set
- Use the trained tagging model too annotate new texts

## 2. Shallow Parsing

As we have already mentioned, [Shallow Parsing](https://en.wikipedia.org/wiki/Shallow_parsing) is a kind of Sequence Labeling. The main difference from Sequence Labeling task, such as Part-of-Speech tagging, where there is an output label (tag) per token; Shallow Parsing additionally performs __chunking__ -- segmentation of input sequence into constituents. Chunking is required to identify categories (or types) of *multi-word expressions*.
In other words, we want to be able to capture information that expressions like `"New York"` that consist of 2 tokens, constitute a single unit.

What this means in practice is that Shallow Parsing performs *jointly* (or not) 2 tasks:
- __Segmentation__ of input into constituents (__spans__)
- __Classification__ (Categorization, Labeling) of these constituents into predefined set of labels (__types__)


## 3. Encoding Segmentation Information: CoNLL Corpus Format

Corpus in CoNLL format consists of series of sentences, separated by blank lines. Each sentence is encoded using a table (or "grid") of values, where each line corresponds to a single word, and each column corresponds to an annotation type. 

The set of columns used by CoNLL-style files can vary from corpus to corpus.

```
        Alex       B-PER
        is         O
        going      O
        to         O
        Los        B-LOC
        Angeles    I-LOC
        in         O
        California B-LOC
```

### 3.1. [IOB Scheme](https://en.wikipedia.org/wiki/Inside%E2%80%93outside%E2%80%93beginning_(tagging))

- The notation scheme is used to label *multi-word* spans in token-per-line format.
    - *Los Angeles* is a *LOCATION* concept that has 2 tokens
- Both, prefix and suffix notations are commons: 
    - prefix: __B-LOC__
    - suffix: __LOC-B__

- Meaning of Prefixes
    - __B__ for (__B__)eginning of span
    - __I__ for (__I__)nside of span
    - __O__ for (__O__)utside of span (no prefix or suffix, just `O`)

#### Alternative Schemes:
- No prefix or suffix (useful when there are no *multi-word* concepts)
```
        Alex       PER
        is         O
        going      O
        to         O
        Los        LOC
        Angeles    LOC
        in         O
        California LOC
```
- __IOB/IOB2/BIO__

- __IOBE__
    - IOB + 
    - __E__ for (__E__)nd of span (or __L__ for (__L__)ast)
```
        Alex       B-PER
        is         O
        going      O
        to         O
        Los        B-LOC
        Angeles    E-LOC
        in         O
        California B-LOC
```
    
- __BILOU/BIOES__
    - IOB + 
    - __L__ for (__L__)ast word of span
    - __U__ for (__U__)nit word (or __S__ for (__S__)ingleton)
```
        Alex       U-PER
        is         O
        going      O
        to         O
        Los        B-LOC
        Angeles    E-LOC
        in         O
        California U-LOC
```

#### Choice of Scheme
- It is possible to convert IOB, IOBE, & BILOU formats to each other
- Each prefix is applied to every concept label, consequently we increase the number of transitions whose probabilities we need to estimate; 
    - increasing data sparseness, as for each label we will have less observations
- The choice of scheme depends on the amount of available data:
    - __IOB__ for least amount
    - __BILOU__ for the most amount 

#### Terminology
There is no strict naming convention regarding schemes (see alternatives) or how each constituent is termed. 
Below is the terminology used in this notebook. 

```
        Alex       B-PER
        is         O
        going      O
        to         O
        Los        B-LOC
        Angeles    I-LOC
        in         O
        California B-LOC
```

##### Interpretation
Segmentation and Labeling data formats encode the following information:
- in string (sentence) `"Alex is going to Los Angeles in California"`
- there are 3 __entities__ (a.k.a. chunks, concepts or slots, depending on NLP task and perspective), that have __types__ (labels)
    - `PERSON`
    - `LOCATION`
    
- entity of __type__ `PERSON`: 
    - has __span__:
        - as tokens from `0` for *CoNLL*: `[0:1]`
    - has __value__: `"Alex"`
        - string *covered by* (*on included*) in __span__
 
*CoNLL* format encodes __tokenization__ informations. In other words, how string `"Alex is going to Los Angeles in California"` is split into tokens. Since most Sequence Labeling algorithms operate on token level, internally the strings are split into tokens, applying *IOB*-like schemes.

#### Exercise
- Wirte the IOB tags for the sentence `"Steve Jobs established Apple Inc."`
    - 'Steve Jobs' is PER
    - 'Apple Inc' is ORG
#### Solution
    - 'Steve' is B-PER
    - 'Jobs' is I-PER
    - 'established' is 0
    - 'Apple' is B-ORG
    - 'Inc' is I-ORG

## 4. Named Entity Recognition with NLTK
[NLTK](https://www.nltk.org/api/nltk.tag.html) provides implementations of popular sequence labeling algorithms for Part-of-Speech Tagging (including [HMM](https://www.nltk.org/api/nltk.tag.html#module-nltk.tag.hmm)), that can be used for Sequence Labeling in general. 

- Loading & working with CoNLL format corpora in NLTK
- Tagger training & testing (running)

To have a custom tagger that labels input text with our __custom label set__, we need to __train__ it on a corpus annotated with this __custom label set__.

Addtionally, NLTK provides [Chunking](http://www.nltk.org/api/nltk.chunk.html). 

### 4.1. NLTK Pre-trained NE Chunker

NLTK provides a classifier that has already been trained to recognize named entities, accessed with the function `nltk.ne_chunk()`. If we set the parameter `binary=True`, then named entities are just tagged as `NE`; otherwise, the classifier adds category labels such as `PERSON`, `ORGANIZATION`, and `GPE`.

In [1]:
import nltk
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping chunkers\maxent_ne_chunker.zip.
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [2]:
from nltk.corpus import treebank

for s in treebank.tagged_sents():
    print(s)
    print(nltk.ne_chunk(s))
    print(nltk.ne_chunk(s, binary=True))
    break

[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]
(S
  (PERSON Pierre/NNP)
  (ORGANIZATION Vinken/NNP)
  ,/,
  61/CD
  years/NNS
  old/JJ
  ,/,
  will/MD
  join/VB
  the/DT
  board/NN
  as/IN
  a/DT
  nonexecutive/JJ
  director/NN
  Nov./NNP
  29/CD
  ./.)
(S
  (NE Pierre/NNP Vinken/NNP)
  ,/,
  61/CD
  years/NNS
  old/JJ
  ,/,
  will/MD
  join/VB
  the/DT
  board/NN
  as/IN
  a/DT
  nonexecutive/JJ
  director/NN
  Nov./NNP
  29/CD
  ./.)


### 4.2. Training NLTK Taggers

In [3]:
nltk.download('conll2002')

[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\adnan\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\conll2002.zip.


True

In [4]:
from nltk.corpus import conll2002

print(len(conll2002.tagged_sents()))
print(conll2002._chunk_types)
print(conll2002.sents('esp.train')[0])
print(conll2002.tagged_sents('esp.train')[0])
print(conll2002.chunked_sents('esp.train')[0])
print(conll2002.iob_sents('esp.train')[0])

35651
('LOC', 'PER', 'ORG', 'MISC')
['Melbourne', '(', 'Australia', ')', ',', '25', 'may', '(', 'EFE', ')', '.']
[('Melbourne', 'NP'), ('(', 'Fpa'), ('Australia', 'NP'), (')', 'Fpt'), (',', 'Fc'), ('25', 'Z'), ('may', 'NC'), ('(', 'Fpa'), ('EFE', 'NC'), (')', 'Fpt'), ('.', 'Fp')]
(S
  (LOC Melbourne/NP)
  (/Fpa
  (LOC Australia/NP)
  )/Fpt
  ,/Fc
  25/Z
  may/NC
  (/Fpa
  (ORG EFE/NC)
  )/Fpt
  ./Fp)
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [5]:
# training hmm on training data: exactly as above
import nltk.tag.hmm as hmm

hmm_model = hmm.HiddenMarkovModelTrainer()

print(conll2002.iob_sents('esp.train')[0])

# let's get only word and iob-tag
trn_sents = [[(text, iob) for text, pos, iob in sent] for sent in conll2002.iob_sents('esp.train')]
print(trn_sents[0])

tst_sents = [[(text, iob) for text, pos, iob in sent] for sent in conll2002.iob_sents('esp.testa')]

hmm_ner = hmm_model.train(trn_sents)
    
# evaluation
accuracy = hmm_ner.accuracy(tst_sents)

print("Accuracy: {:6.4f}".format(accuracy))

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]
[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:334: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:332: RuntimeWarning: overflow encountered in cast
  P[i] = self._priors.logprob(si)
C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


Accuracy: 0.3760


##### NLTK Chunk Tagger Note
HMM uses only words as input, NLTK also povides trainable MaxEnt Chunker Tagger, which unfortunatelly requires `megam` file. Unfortunatelly, it is very convoluted to install. (http://www.umiacs.umd.edu/~hal/megam/index.html)

### Exercise

#### Segmentation 
Train a tagger to perform *segmentation* of input sentences into constituents
- Strip concept information from output labels (i.e. keep only IOB-prefix)
- Train tagger to predict segmentation labels
- Evaluate segmentation performance

In [9]:
import nltk.tag.hmm as hmm

hmm_model = hmm.HiddenMarkovModelTrainer()

print(conll2002.iob_sents('esp.train')[0])

# let's get only word and iob-tag
# Hint: split iob with using '-'
trn_sents_seg = [[(text, iob.split('-')[0]) for text, pos, iob in sent] for sent in conll2002.iob_sents('esp.train')]
print(trn_sents_seg[0])
tst_sents_seg = [[(text, iob.split('-')[0]) for text, pos, iob in sent] for sent in conll2002.iob_sents('esp.testa')]
print(tst_sents_seg[0])

# Train tagger to predict segmentation labels
hmm_seg = hmm_model.train(trn_sents_seg)

# evaluation
accuracy = hmm_seg.accuracy(tst_sents_seg)

print("Accuracy: {:6.4f}".format(accuracy))

[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]
[('Melbourne', 'B'), ('(', 'O'), ('Australia', 'B'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B'), (')', 'O'), ('.', 'O')]
[('Sao', 'B'), ('Paulo', 'I'), ('(', 'O'), ('Brasil', 'B'), (')', 'O'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B'), (')', 'O'), ('.', 'O')]


C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:336: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])
C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:334: RuntimeWarning: overflow encountered in cast
  X[i, j] = self._transitions[si].logprob(self._states[j])
C:\Users\adnan\anaconda3\envs\NLU-LABs-venv\lib\site-packages\nltk\tag\hmm.py:364: RuntimeWarning: overflow encountered in cast
  O[i, k] = self._output_logprob(si, self._symbols[k])


Accuracy: 0.4297


#### CoNLL Eval
CoNLL Community developed a perl script to evaluate *segmentation* and *labeling* performance jointly using IOB information. Such evaluation provides more accurate assessment of the shallow parsing performance, in comparison to token-level metrics (e.g. NLTK accuracy).

- import `evaluate` function from `conll.py` (example shown)
- evaluate tagger predictions
- compare performances to token-level accuracies

In [10]:
# to import conll
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

from conll import evaluate
# for nice tables
import pandas as pd

# getting references (note that it is testb this time)
refs = [[(text, iob) for text, pos, iob in sent] for sent in conll2002.iob_sents('esp.testb')]
print(refs[0])
# getting hypotheses
hyps = [hmm_ner.tag(s) for s in conll2002.sents('esp.testb')]
print(hyps[0])

results = evaluate(refs, hyps)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')]
[('La', 'B-LOC'), ('Coruña', 'I-LOC'), (',', 'O'), ('23', 'O'), ('may', 'O'), ('(', 'O'), ('EFECOM', 'B-ORG'), (')', 'O'), ('.', 'O')]


,p,r,f,s
PER,0.691,0.280,0.399,735
ORG,0.795,0.396,0.528,1400
MISC,0.570,0.203,0.299,340
LOC,0.030,0.849,0.058,1084
total,0.055,0.491,0.098,3559


## 5. Named Entity Recognition with spaCy

In [11]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

txt = 'Pierre Vinken, 61 years old, will join the board as a nonexecutive director Nov. 29.'
doc = nlp(txt)

print([ent.text for ent in doc.ents])
print([(t.ent_type_, t.ent_iob_) for t in doc])

['Pierre Vinken', '61 years old', 'Nov. 29']
[('PERSON', 'B'), ('PERSON', 'I'), ('', 'O'), ('DATE', 'B'), ('DATE', 'I'), ('DATE', 'I'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('', 'O'), ('DATE', 'B'), ('DATE', 'I'), ('', 'O')]


## 6. Shallow Parsing with Conditional Random Fields

[CRFs](https://en.wikipedia.org/wiki/Conditional_random_field) are a type of __discriminative undirected probabilistic graphical model__. 
It is a generalization of __any__ undirected graph structure.
In Natural Language Processing, the structure is a *sequence* of words, and conditioning is on *previous transition*. This is known as __Linear Chain CRFs__.

For general graphs, the problem of exact inference in CRFs is intractable. For __Linear Chain CRFs__, however, there is an exact solution, and the used algorithm is "analogous to the forward-backward and Viterbi algorithm for the case of Hidden Markov Models". 

### Python CRF Tutorials

Authors of the python packages provide tutorials in the form of notebooks already!

__Follow the tutorials to learn the tools.__

- [sklearn-crfsuite notebook](https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb)
- [python-crfsuite notebook](https://github.com/scrapinghub/python-crfsuite/blob/master/examples/CoNLL%202002.ipynb)

- `bias` is explained [here](https://github.com/scrapinghub/python-crfsuite/issues/73)

### Baseline

Let's prepare a CRF baseline for our dataset that:
- considers only word itself and the previous tag (similar to HMM).

In [64]:
# data set
print(len(trn_sents))
print(len(tst_sents))
print(trn_sents[0])

8323
1915
[('Melbourne', 'B-LOC'), ('(', 'O'), ('Australia', 'B-LOC'), (')', 'O'), (',', 'O'), ('25', 'O'), ('may', 'O'), ('(', 'O'), ('EFE', 'B-ORG'), (')', 'O'), ('.', 'O')]


Let's copy & re-define feature extraction functions from the tutorials.

In [65]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [66]:
def word2features(sent, i):
    word = sent[i][0]
    return {'bias': 1.0, 'word.lower()': word.lower()}

Let's inspect our baseline features.

In [67]:
sent2features(trn_sents[0])[0]

{'bias': 1.0, 'word.lower()': 'melbourne'}

#### Feature Extraction 

In [68]:
%%time
trn_feats = [sent2features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

CPU times: total: 469 ms
Wall time: 773 ms


#### Training

In [17]:
from sklearn_crfsuite import CRF

crf = CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

In [18]:
%%time
# workaround for scikit-learn 1.0
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 12.3 s
Wall time: 13.7 s


#### Prediction

In [19]:
tst_feats = [sent2features(s) for s in tst_sents]
pred = crf.predict(tst_feats)

#### Evaluation

We are going to use our `conll` evaluation script. (Notice that tools report token level metrics.)

For that we will need to modify our prediction output a bit, to make it a tuple.

In [20]:
print(pred[0])

['B-LOC', 'I-LOC', 'O', 'B-LOC', 'O', 'O', 'O', 'O', 'O', 'B-ORG', 'O', 'O']


In [21]:
hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

In [22]:
results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.859,0.375,0.522,1222
ORG,0.729,0.442,0.551,1700
MISC,0.542,0.261,0.352,445
LOC,0.699,0.713,0.706,985
total,0.729,0.466,0.569,4352


## 7. Feature Engineering

One of the strengths of CRFs lies in its ability to make use of rich feature representation. The process of extracting features from raw data is know as [feature engineering](https://en.wikipedia.org/wiki/Feature_engineering).

Common features used in sequence labeling with CRFs are:
- part-of-speech tags
- lemmas
- token character prefixes and suffixes (e.g. first and last 1, 2, 3 characters of a word; `word[-3:]` in tutorial is suffix of length 3).

### 7.1. SpaCy Token Features

[spaCy](https://spacy.io/) provides a convenient way to augment our feature set with common features using in Natural Language Processing. 

The list of provided token-level features is available [here](https://spacy.io/api/token#attributes).

### 7.2. Adding Features to CRF

Let's modify `sent2features` function to make use of spaCy features.

In [61]:
# get Spanish model of spacy
!pip install es_core_news_sm

In [69]:
import spacy
from spacy.tokenizer import Tokenizer
import es_core_news_sm
nlp = es_core_news_sm.load()

# nlp = spacy.load("es_core_news_sm")
nlp.tokenizer = Tokenizer(nlp.vocab)  # to use white space tokenization (generally a bad idea for unknown data)

def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for token in spacy_sent:
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'pos': token.pos_,
            'lemma': token.lemma_
        }
        feats.append(token_feats)
    
    return feats

In [70]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]
tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [71]:
print(trn_feats[0])

[{'bias': 1.0, 'word.lower()': 'melbourne', 'pos': 'PROPN', 'lemma': 'Melbourne'}, {'bias': 1.0, 'word.lower()': '(', 'pos': 'PUNCT', 'lemma': '('}, {'bias': 1.0, 'word.lower()': 'australia', 'pos': 'PROPN', 'lemma': 'Australia'}, {'bias': 1.0, 'word.lower()': ')', 'pos': 'PUNCT', 'lemma': ')'}, {'bias': 1.0, 'word.lower()': ',', 'pos': 'PUNCT', 'lemma': ','}, {'bias': 1.0, 'word.lower()': '25', 'pos': 'NUM', 'lemma': '25'}, {'bias': 1.0, 'word.lower()': 'may', 'pos': 'NOUN', 'lemma': 'may'}, {'bias': 1.0, 'word.lower()': '(', 'pos': 'PUNCT', 'lemma': '('}, {'bias': 1.0, 'word.lower()': 'efe', 'pos': 'PROPN', 'lemma': 'EFE'}, {'bias': 1.0, 'word.lower()': ')', 'pos': 'PUNCT', 'lemma': ')'}, {'bias': 1.0, 'word.lower()': '.', 'pos': 'PUNCT', 'lemma': '.'}]


In [72]:
crf = CRF(
    algorithm='lbfgs', 
    c1=0.1, 
    c2=0.1, 
    max_iterations=100, 
    all_possible_transitions=True
)

In [73]:
%%time
# workaround for scikit-learn 1.0
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 18 s
Wall time: 18.1 s


In [74]:
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

In [75]:
results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.820,0.649,0.725,1222
ORG,0.606,0.725,0.660,1700
MISC,0.623,0.427,0.507,445
LOC,0.723,0.735,0.729,985
total,0.683,0.676,0.679,4352


## Lab Exercise

The exsecise is to experiment with a CRF model on NER task. You have to train and test a CRF with using different features on the conll2002 corpus (the same that we used here in the lab). 

The features that you have to experiment with are:
- Baseline using the fetures in sent2spacy_features
    - Train the model and print results on the test set
- Add the "suffix" feature
    - Train the model and print results on the test set
- Add all the features used in the [tutorial on CoNLL dataset](https://github.com/TeamHG-Memex/sklearn-crfsuite/blob/master/docs/CoNLL2002.ipynb)
    - Train the model and print results on the test set
- Increase the feature window (number of previous and next token) to:
    - `[-1, +1]`
        - Train the model and print results on the test set
    - `[-2, +2]`
        - Train the model and print results on the test set

        
The format of the results has to be the table that we used so far, that is:
```python
results = evaluate(tst_sents, hyp)
pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)
```


### Experimenting with a CRF model on NER task.
Train and test a CRF with using different features on the conll2002 corpus

### Importing libraries

In [109]:
# to import conll
import os
import sys
sys.path.insert(0, os.path.abspath('../src/'))

from conll import evaluate
# for nice tables
import pandas as pd

In [110]:
# get Spanish model of spacy
!pip install es_core_news_sm

### 1. Baseline using the features in sent2spacy_features

In [119]:
import spacy
from spacy.tokenizer import Tokenizer
import es_core_news_sm

nlp = es_core_news_sm.load()

# nlp = spacy.load("es_core_news_sm")
nlp.tokenizer = Tokenizer(nlp.vocab)  # to use white space tokenization (generally a bad idea for unknown data)


def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for token in spacy_sent:
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'pos': token.pos_,
            'lemma': token.lemma_
        }
        feats.append(token_feats)

    return feats

In [120]:
def sent2labels(sent):
    return [label for token, label in sent]

def sent2tokens(sent):
    return [token for token, label in sent]

In [121]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [122]:
# train the model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [123]:
%%time
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 12.8 s
Wall time: 13.6 s


In [124]:
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.820,0.649,0.725,1222
ORG,0.606,0.725,0.660,1700
MISC,0.623,0.427,0.507,445
LOC,0.723,0.735,0.729,985
total,0.683,0.676,0.679,4352


### 2. Add the "suffix" feature

#### 2.1. Add the "suffix" feature to sent2spacy_features

In [125]:
def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for token in spacy_sent:
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'pos': token.pos_,
            'lemma': token.lemma_,
            'suffix': token.lower_[-3:]
        }
        feats.append(token_feats)

    return feats

In [126]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [127]:
# train the model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [128]:
%%time
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 14 s
Wall time: 14.7 s


In [129]:
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.827,0.679,0.746,1222
ORG,0.684,0.679,0.682,1700
MISC,0.620,0.436,0.512,445
LOC,0.656,0.768,0.707,985
total,0.706,0.674,0.690,4352


### 3. Add all the features used in the tutorial on CoNLL dataset

In [143]:
# define the features in sent2spacy_features
def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for token in spacy_sent:
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'word[-3:]': token.lower_[-3:],
            'word[-2:]': token.lower_[-2:],
            'word.isupper()': token.is_upper,
            'word.istitle()': token.is_title,
            'word.isdigit()': token.is_digit,
            'postag': token.pos_,
            'postag[:2]': token.pos_[:2],
            'lemma': token.lemma_,
            'suffix': token.lower_[-1:],
            'suffix2': token.lower_[-2:],
            'suffix3': token.lower_[-3:]
        }
        feats.append(token_feats)

    return feats

In [144]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [145]:
trn_feats[0]

[{'bias': 1.0,
  'word.lower()': 'melbourne',
  'word[-3:]': 'rne',
  'word[-2:]': 'ne',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PROPN',
  'postag[:2]': 'PR',
  'lemma': 'Melbourne',
  'suffix': 'e',
  'suffix2': 'ne',
  'suffix3': 'rne'},
 {'bias': 1.0,
  'word.lower()': '(',
  'word[-3:]': '(',
  'word[-2:]': '(',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'PUNCT',
  'postag[:2]': 'PU',
  'lemma': '(',
  'suffix': '(',
  'suffix2': '(',
  'suffix3': '('},
 {'bias': 1.0,
  'word.lower()': 'australia',
  'word[-3:]': 'lia',
  'word[-2:]': 'ia',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PROPN',
  'postag[:2]': 'PR',
  'lemma': 'Australia',
  'suffix': 'a',
  'suffix2': 'ia',
  'suffix3': 'lia'},
 {'bias': 1.0,
  'word.lower()': ')',
  'word[-3:]': ')',
  'word[-2:]': ')',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.is

In [146]:
# train the model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [147]:
%%time
# workaround for scikit-learn 1.0
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 22.4 s
Wall time: 23.3 s


In [148]:
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.834,0.732,0.780,1222
ORG,0.751,0.688,0.718,1700
MISC,0.588,0.490,0.534,445
LOC,0.635,0.771,0.696,985
total,0.725,0.699,0.712,4352


### 4. Increase the feature window (number of previous and next token) to: [-1, +1]

In [149]:
# define the features in sent2spacy_features
def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for i, token in enumerate(spacy_sent):
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'word[-3:]': token.lower_[-3:],
            'word[-2:]': token.lower_[-2:],
            'word.isupper()': token.is_upper,
            'word.istitle()': token.is_title,
            'word.isdigit()': token.is_digit,
            'postag': token.pos_,
            'postag[:2]': token.pos_[:2],
            'lemma': token.lemma_,
            'suffix': token.lower_[-1:],
            'suffix2': token.lower_[-2:],
            'suffix3': token.lower_[-3:]
        }
        if i > 0:
            token1 = spacy_sent[i-1]
            token_feats.update({
                '-1:word.lower()': token1.lower_,
                '-1:word.istitle()': token1.is_title,
                '-1:word.isupper()': token1.is_upper,
                '-1:postag': token1.pos_,
                '-1:postag[:2]': token1.pos_[:2],
                '-1:lemma': token1.lemma_,
                '-1:suffix': token1.lower_[-1:],
                '-1:suffix2': token1.lower_[-2:],
                '-1:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['BOS'] = True

        if i < len(spacy_sent)-1:
            token1 = spacy_sent[i+1]
            token_feats.update({
                '+1:word.lower()': token1.lower_,
                '+1:word.istitle()': token1.is_title,
                '+1:word.isupper()': token1.is_upper,
                '+1:postag': token1.pos_,
                '+1:postag[:2]': token1.pos_[:2],
                '+1:lemma': token1.lemma_,
                '+1:suffix': token1.lower_[-1:],
                '+1:suffix2': token1.lower_[-2:],
                '+1:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['EOS'] = True

        feats.append(token_feats)

    return feats

In [152]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [153]:
trn_feats[0]

[{'bias': 1.0,
  'word.lower()': 'melbourne',
  'word[-3:]': 'rne',
  'word[-2:]': 'ne',
  'word.isupper()': False,
  'word.istitle()': True,
  'word.isdigit()': False,
  'postag': 'PROPN',
  'postag[:2]': 'PR',
  'lemma': 'Melbourne',
  'suffix': 'e',
  'suffix2': 'ne',
  'suffix3': 'rne',
  'BOS': True,
  '+1:word.lower()': '(',
  '+1:word.istitle()': False,
  '+1:word.isupper()': False,
  '+1:postag': 'PUNCT',
  '+1:postag[:2]': 'PU',
  '+1:lemma': '(',
  '+1:suffix': '(',
  '+1:suffix2': '(',
  '+1:suffix3': '('},
 {'bias': 1.0,
  'word.lower()': '(',
  'word[-3:]': '(',
  'word[-2:]': '(',
  'word.isupper()': False,
  'word.istitle()': False,
  'word.isdigit()': False,
  'postag': 'PUNCT',
  'postag[:2]': 'PU',
  'lemma': '(',
  'suffix': '(',
  'suffix2': '(',
  'suffix3': '(',
  '-1:word.lower()': 'melbourne',
  '-1:word.istitle()': True,
  '-1:word.isupper()': False,
  '-1:postag': 'PROPN',
  '-1:postag[:2]': 'PR',
  '-1:lemma': 'Melbourne',
  '-1:suffix': 'e',
  '-1:suffix2': 

In [154]:
# train the model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [155]:
%%time
# workaround for scikit-learn 1.0
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 44.9 s
Wall time: 45.7 s


In [156]:
# predict
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.874,0.787,0.828,1222
ORG,0.776,0.737,0.756,1700
MISC,0.559,0.470,0.510,445
LOC,0.649,0.797,0.716,985
total,0.747,0.737,0.742,4352


### 5. Increase the feature window (number of previous and next token) to: [-2, +2]

In [157]:
# define the features in sent2spacy_features
def sent2spacy_features(sent):
    spacy_sent = nlp(" ".join(sent2tokens(sent)))
    feats = []
    for i, token in enumerate(spacy_sent):
        token_feats = {
            'bias': 1.0,
            'word.lower()': token.lower_,
            'word[-3:]': token.lower_[-3:],
            'word[-2:]': token.lower_[-2:],
            'word.isupper()': token.is_upper,
            'word.istitle()': token.is_title,
            'word.isdigit()': token.is_digit,
            'postag': token.pos_,
            'postag[:2]': token.pos_[:2],
            'lemma': token.lemma_,
            'suffix': token.lower_[-1:],
            'suffix2': token.lower_[-2:],
            'suffix3': token.lower_[-3:]
        }
        if i > 0:
            token1 = spacy_sent[i-1]
            token_feats.update({
                '-1:word.lower()': token1.lower_,
                '-1:word.istitle()': token1.is_title,
                '-1:word.isupper()': token1.is_upper,
                '-1:postag': token1.pos_,
                '-1:postag[:2]': token1.pos_[:2],
                '-1:lemma': token1.lemma_,
                '-1:suffix': token1.lower_[-1:],
                '-1:suffix2': token1.lower_[-2:],
                '-1:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['BOS'] = True

        if i < len(spacy_sent)-1:
            token1 = spacy_sent[i+1]
            token_feats.update({
                '+1:word.lower()': token1.lower_,
                '+1:word.istitle()': token1.is_title,
                '+1:word.isupper()': token1.is_upper,
                '+1:postag': token1.pos_,
                '+1:postag[:2]': token1.pos_[:2],
                '+1:lemma': token1.lemma_,
                '+1:suffix': token1.lower_[-1:],
                '+1:suffix2': token1.lower_[-2:],
                '+1:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['EOS'] = True

        if i > 1:
            token1 = spacy_sent[i-2]
            token_feats.update({
                '-2:word.lower()': token1.lower_,
                '-2:word.istitle()': token1.is_title,
                '-2:word.isupper()': token1.is_upper,
                '-2:postag': token1.pos_,
                '-2:postag[:2]': token1.pos_[:2],
                '-2:lemma': token1.lemma_,
                '-2:suffix': token1.lower_[-1:],
                '-2:suffix2': token1.lower_[-2:],
                '-2:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['BOS'] = True

        if i < len(spacy_sent)-2:
            token1 = spacy_sent[i+2]
            token_feats.update({
                '+2:word.lower()': token1.lower_,
                '+2:word.istitle()': token1.is_title,
                '+2:word.isupper()': token1.is_upper,
                '+2:postag': token1.pos_,
                '+2:postag[:2]': token1.pos_[:2],
                '+2:lemma': token1.lemma_,
                '+2:suffix': token1.lower_[-1:],
                '+2:suffix2': token1.lower_[-2:],
                '+2:suffix3': token1.lower_[-3:]
            })
        else:
            token_feats['EOS'] = True

        feats.append(token_feats)

    return feats

In [158]:
trn_feats = [sent2spacy_features(s) for s in trn_sents]
trn_label = [sent2labels(s) for s in trn_sents]

tst_feats = [sent2spacy_features(s) for s in tst_sents]

In [159]:
# train the model
crf = CRF(
    algorithm='lbfgs',
    c1=0.1,
    c2=0.1,
    max_iterations=100,
    all_possible_transitions=True
)

In [160]:
%%time
try:
    crf.fit(trn_feats, trn_label)
except AttributeError:
    pass

CPU times: total: 1min 11s
Wall time: 1min 12s


In [161]:
# predict
pred = crf.predict(tst_feats)

hyp = [[(tst_feats[i][j], t) for j, t in enumerate(tokens)] for i, tokens in enumerate(pred)]

results = evaluate(tst_sents, hyp)

pd_tbl = pd.DataFrame().from_dict(results, orient='index')
pd_tbl.round(decimals=3)

,p,r,f,s
PER,0.871,0.792,0.830,1222
ORG,0.781,0.747,0.763,1700
MISC,0.543,0.472,0.505,445
LOC,0.668,0.805,0.730,985
total,0.751,0.745,0.748,4352


### Conclusion